# Load the data

In [1]:
import pandas as pd
from src.data.data import remove_fake_reps, retrieve_variables, combine_variables
from src.features.features import aggregate_vars

Two ways to go : 
- Make sure that the data are in path_to_data following either bids (home) or pseudo-bids (scans) convention. This is done by running the next cell.
- Download the file at --ask me for link-- and open it with the following cell

In [ ]:
path_to_data = '/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data'

# load the data in a dict structure
alldata_dict = {}
for subject in ['sub-01', 'sub-02']:
    for level in ['1', '4', '5']:
        for setup in ['home', 'scan']:
            if setup == 'home':
                behav = True
            if setup == 'scan':
                behav = False
            allvars = combine_variables(path_to_data, subject, level, behav=behav, save=True)
            allvars_clean = remove_fake_reps(allvars)
            data_dict = aggregate_vars(allvars, metric=None, days_of_train=False, health_lost=True, max_score=True, completion_perc=True)
            alldata_dict['{}_{}_{}'.format(subject, level, setup)] = data_dict
            

# arrange the data as a dataframe suited for stats
df = pd.DataFrame(columns=['Passage order', 'Health loss', 'Max score', 'Percent complete', 'Subject', 'Level', 'Setup'])
for key in alldata_dict.keys():
    for idx in alldata_dict[key][list(alldata_dict[key].keys())[0]]:
        df = df.append({
         "Passage order": alldata_dict[key]['Passage order'][idx],
         "Health loss":  alldata_dict[key]['Health loss'][idx],
         "Max score": alldata_dict[key]['Max score'][idx],
         "Percent complete": alldata_dict[key]['Percent complete'][idx],
         "Subject": key[5],
         "Level": key[7],
         "Setup": key[9:13]
          }, ignore_index=True)

df.reset_index().to_csv(path_to_data + '/processed/homeVSscan.csv')

/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data/bidsbehav/sub-01/ses-001/beh/sub-01_ses-001_task-shinobi_level-1_rep-001_gamedata.bk2
/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data/bidsbehav/sub-01/ses-002/beh/sub-01_ses-002_task-shinobi_level-1_rep-001_gamedata.bk2
/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data/bidsbehav/sub-01/ses-002/beh/sub-01_ses-002_task-shinobi_level-1_rep-002_gamedata.bk2
/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data/bidsbehav/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-001_gamedata.bk2
/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data/bidsbehav/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-002_gamedata.bk2
/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data/bidsbehav/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-003_gamedata.bk2
/home/hyruuk/GitHub/neuromod/hyruuk_shinobi_behav/data/bidsbehav/sub-01/ses-003/beh/sub-01_ses-003_task-shinobi_level-1_rep-004_gamedata.bk2
/home/hyruuk/

In [ ]:
df = pd.read_csv(path_to_data + '/processed/homeVSscan.csv')

# Now let's run some stats

We want to run an ANOVA on each measure with the factors "Setup" (home vs scan), "Subject" (1 vs 2, maybe more later), "Level" (1 vs 4 vs 5), and we would like to see the Setup factor as non-significant.